In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import StratifiedKFold
from scipy.io import arff
from sklearn.naive_bayes import MultinomialNB,GaussianNB


def loadReutersData(documents,labels):
    categories_list=['acq','crude','earn','grain','interest','money-fx','ship','trade']
    docCount=0
    for i in range(0,len(categories_list)):
        category_docs = reuters.fileids(categories_list[i])
        print (categories_list[i])
        for document_id in reuters.fileids(categories_list[i]):
            if(len(reuters.categories(document_id))==1):
                content=str(reuters.raw(document_id))
                soup = BeautifulSoup(content)
                content=soup.get_text()
                documents.append(content)
                docCount+=1
                labels.append(str(reuters.categories(document_id)))



def most_common(lst):
    return max(set(lst), key=lst.count)
def tokenize1(documents):
    tokens=[]
    content= documents
    tokens=(word_tokenize(content))
    tokens= [token.lower() for token in tokens ]
    tokens = [token for token in tokens if token not in stopwords]
    tokens= [token for token in tokens if token.isalpha()]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens= [token for token in tokens if len(token)>3 ]
    return tokens
def count_values_in_range(a, range_min, range_max):

    # "between" returns a boolean Series equivalent to left <= series <= right.
    # NA values will be treated as False.
    return ((range_min <= a) & (a <= range_max)).sum()
def display_scores(vectorizer, tfidf_result):
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
def sorted_tfs(tfs,n):
    doc,terms=tfs.shape
    ind=(np.argsort(-(np.asarray(tfs.sum(axis=0)).ravel())))
    scores=np.zeros((doc,n))
    for i in range(0,len(documents)):
        for j in range(0,n):
            if(tfs[i,ind[j]]!=0):
                scores[i,j]=tfs[i,ind[j]]
    return scores

def dislay_tfidf(vectorizer,tfidf_result):
    print(vectorizer.get_feature_names())
    print(tfidf_result)
def Manhattan(doc1,doc2):
    return distance.cityblock(doc1,doc2)                       
def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def Jaccard(a, b):#distance
    return distance.jaccard(a,b)

def bhatta(a,b):

    length=len(a)
    score = 0;
    score=np.sum(np.sqrt( np.multiply(a,b) ))
    distance=-1*np.log(score)
    return distance;  

def EPDSM(doc1,doc2):
    a=set(np.nonzero(doc1!=0)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2!=0)[0])  #indices of non zero elements in doc2
    intersection=np.sum(np.minimum(doc1,doc2))
    union=np.sum(np.maximum(doc1,doc2))
    PF=len(a.intersection(b))
    M=len(doc1)
    AF=M-len(a.union(b))
    psdm=(intersection/union)*((PF+1)/(M-AF+1))
    return psdm

def EPairwise(a,b):
    percentage=1 #100 percent
    K1=np.count_nonzero(a)
    K2=np.count_nonzero(b)
    k=percentage*min(K1,K2)
    setA=set(np.argpartition(a, len(a)-1 - k)[-k:])
    setB=set(np.argpartition(b, len(b) -1- k)[-k:])
    union=np.array(list(setA.union(setB)))
    elementsA=a[union]
    elementsB=b[union]
    return distance.cosine(elementsA, elementsB)


def Pairwise(a,b):
    percentage=1 #100 percent
    K1=np.count_nonzero(a)
    K2=np.count_nonzero(b)
    k=percentage*min(K1,K2)
   # setA=set((-a).argsort()[:k])
   # setB=set((-b).argsort()[:k])
    
    setA=set(np.argpartition(a, len(a)-1 - k)[-k:])
    setB=set(np.argpartition(b, len(b)-1 - k)[-k:])
    union=setA.union(setB)
    elementsA=[a[ind] for ind in union]
    elementsB=[b[ind] for ind in union]
    dist= distance.cosine(elementsA, elementsB)
    return dist

def Dice(a, b):
    a=[(a.astype(bool)).astype(int)]
    b=[(b.astype(bool)).astype(int)]
    return distance.dice(a,b)

def EDice(doc1,doc2):
    a=set(list(np.nonzero(doc1!=0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2!=0)[0]))  #indices of non zero elements in doc2
    intr=len(a.intersection(b))
    aComp=len(a-b)
    bComp=len(b-a)
    sim=0
    if (intr+aComp+bComp)!=0:
        sim=2*(intr/(2*(intr+aComp+bComp)))
    return sim

def smtp(doc1,doc2):
    lemda=0.0001
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=a.intersection(b) # indices where both docs have non zero elements
    union=a.union(b) # indices where either doc has non zero elements
    d1=np.array(list(a-intersection)) # doc1 !=0 and doc2=0
    d2=np.array(list(b-intersection)) # doc1 =0 and doc2!=0
    doc1=np.array(doc1)
    doc2=np.array(doc2)
    Nstar=0
    intersection=np.array(list(intersection))
    if (len(intersection)>0):
        term1=np.exp(-1*np.square(( doc1[intersection]-doc2[intersection] )/var[intersection]))
        Nstar=sum(0.5* (1+term1)) +lemda* -1 *(len(d1)+len(d2))
    else:
        Nstar=lemda* -1 *(len(d1)+len(d2))   
    Nunion=len(intersection)+len(d1)+len(d2)
    smtp=0
    if Nunion!=0:
        smtp=((Nstar/Nunion)+lemda)/(1+lemda)
    return smtp
def esmtp(doc1,doc2,var):
    lemda=1
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection=a.intersection(b) # indices where both docs have non zero elements
    union=a.union(b) # indices where either doc has non zero elements
    l1=len(a-intersection) # doc1 !=0 and doc2=0
    l2=len(b-intersection) # doc1 =0 and doc2!=0
    Nstar=0
    intersection=np.array(list(intersection))
    if (len(intersection)>0):
        term1=np.exp(-1*np.square(( doc1[intersection]-doc2[intersection] )/var[intersection]))
        Nstar=np.sum(0.5* (1+term1)) +lemda* -1 *(l1+l2)
    else:
        Nstar=lemda* -1 *(l1+l2)   
    Nunion=len(intersection)+l1+l2
    smtp=((Nstar/Nunion)+lemda)/(1+lemda)
    return smtp



def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return isc
def STB_SM_new(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection=np.array(list(a.intersection(b)))
    comp1=a-b
    comp2=b-a   
    comp1=np.array(list(comp1))
    comp2=np.array(list(comp2))
    X,Y,D1,D2,Z1,Z2,sim=0,0,0,0,0,0,0
    if len(intersection)>0:
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
    if len(comp1)>0:
        D1=np.sum(doc1[comp1])
    if len(comp2)>0:
        D2=np.sum(doc2[comp2])
    Z1=np.sum(doc1)
    Z2=np.sum(doc2) 
    if Z1!=0 and Z2!=0:
        sim=((X*Y)/(Z1*Z2))*(1-((D1*D2)/(Z1*Z2)))
    return sim
def ECSM_P1(doc1,doc2):
    simValue=0
    N=len(doc1)#total_features;
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2NSMT
    Nab=len(a.intersection(b))
    F=len(a.union(b))-Nab
    simValue=(1-F/N)
    return simValue
def ECSM_P2(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2NSMT
    sim=0
    if (len(a)+len(b))!=0:
        sim=(2*len(a.intersection(b)))/(len(a)+len(b))
    return sim

def EBLAB_SM(doc1,doc2):
    return (0.5*(ECSM_P1(doc1,doc2)+ECSM_P2(doc1,doc2)))

def PCC(a, b):
    pcc, col=pearsonr(a,b)
    return abs(pcc)
def ESP(doc1,doc2):
    a=set(list(np.nonzero(doc1!=0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2!=0)[0]))  #indices of non zero elements in doc2
    nonzeroTerm=list(a.intersection(b))  #indices where both docs have non zero elemments
    d1=doc1[nonzeroTerm] #doc1 values of doc1 intersection  doc2
    d2=doc2[nonzeroTerm] #doc2 values of doc1 intersection  doc2
    minArr=np.minimum(d1,d2) #minimum values  of d1 and d2
    maxArr=np.maximum(d1,d2) #maximum values  of d1 and d2
    ln=len(nonzeroTerm)
    docCount=np.zeros(ln)
    aData=allData[:,nonzeroTerm] #extracting data with only feature indices of doc1 intersection  doc2
    for ti in range(0,ln):
        docCount[ti]=count_values_in_range(aData[:,ti], minArr[ti], maxArr[ti]) #count values which lies between min and max
    #norm factor which is ist part of equation 6 in paper
    normFactor= len(a.union(b))#norm factor equal to length of doc1 union doc2
    dCount=np.array(docCount[np.nonzero(docCount!=0)[0]])# get all non zero values between min and max
    SP_val=0
    if(normFactor!=0):
        SP_val=np.sum(np.log(totalDocs/dCount)) #2nd part in equation 6 in paper
        SP_val = (SP_val/normFactor) #equation 6 in paper
    return SP_val

class SimMeaure:

    #constructor

    def __init__(self,k = 1,metric="euclidean",termOccurance=None, docOccurance=None):

        
        self.k=k
        #print ("KNN",self.k)
        self.metric=metric
        self.trainingData=[]
        self.trainLabels=[]
        self.smtp=None
        self.termOccurance=termOccurance
        self.docOccurance=docOccurance
         
    
         

    def fit(self, training_data, trainLabels ):
        self.trainingData=training_data
        self.trainLabels=trainLabels
        
            
       
    def predict(self, testData):
        train=self.trainingData
        var=np.zeros(train.shape[1])
        if(self.metric=="smtp" or  self.metric=="esmtp" or  self.metric=="DSM"):
            var=np.var(train,axis=0)
            print("Vrainace is",var)
        distList=["KL","extendedJaccard","bhatta","Euclidean","Cosine","Jaccard","JS","EJS","Dice",
                  "Pairwise","EPairwise","Manhattan","EnhancedJaccard","EEnhancedJaccard","eextendedJaccard","EJS","Minkowski"]#distance
        func=globals()[self.metric]
        predLabel=[]
        kList=[1,3,5,9,15,30,45,70,90,120]
        for kk in range(0,len(kList)):
            temp=[]
            predLabel.append(temp)
            ''''''
        distMatrix=[]         
        for i in(range(0,len(testData))):
            #print(i)
            dist=[]
            for j in(range(0,len(train))):
                if ((not np.any(testData[i])) or (not np.any( train[j]))):
                    if(self.metric in distList):
                        dist.append(-1)
                    else:
                        dist.append(0)
                else:
                    if(self.metric=="smtp" or self.metric=="smtp_improved" or self.metric=="esmtp" or self.metric=="DSM"):
                        dist.append(func(testData[i],train[j],var)) 
                    
                    elif(self.metric=="SP"):
                        doc1=testData[i]
                        doc2=train[j]
                        nonzeroDoc1=set(np.nonzero(doc1)[0])
                        nonzeroDoc2=set(np.nonzero(doc2)[0])
                        nonzeroTerm=list(nonzeroDoc1.intersection(nonzeroDoc2))
                        minArr=np.minimum(doc1[nonzeroTerm],doc2[nonzeroTerm])
                        maxArr=np.maximum(doc1[nonzeroTerm],doc2[nonzeroTerm])
                        ln=len(nonzeroTerm)
                        aData=allData[:,nonzeroTerm]
                        docCount=np.zeros(ln)
                        for ti in range(0,ln):
                            tData=pd.Series(aData[:,ti])
                            docCount[ti]=count_values_in_range(tData, minArr[ti], maxArr[ti])
                        dist.append(SP(testData[i],train[j],docCount,totalDocs))
                    else:
                        dist.append(func(testData[i], train[j]))
            flag=-1
            dist=list(dist)
            if(self.metric in distList):
                dist=[(1/(x+0.0001)) for x in dist]
                flag=1
            distMatrix.append(dist)
        return distMatrix


def classify(metric='Cosine',termOccurance=None, docOccurance=None):
    time1= time.time()
    y_pred =NBCombined(train_data, train_labels,test_data,metric)
    time2= time.time()

    
    for i in range(0,1):
        #Accuracy
    
        accuracy=accuracy_score(test_labels, y_pred)
        
        #Macro Precision
        
        macroP=precision_score(test_labels, y_pred,average='macro')
        
        #Micro Precision
        
        microP=precision_score(test_labels, y_pred,average='micro')
        
        #Macro Recall
        
        macroR=recall_score(test_labels, y_pred,average='macro')
        
        #Micro Recall
        
        microR=recall_score(test_labels, y_pred,average='micro')
        
        
        fMeasure=f1_score(test_labels, y_pred,average='macro')
       # PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=fMeasure)
        
        m_fMeasure=f1_score(test_labels, y_pred,average='micro')
       # PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=fMeasure)
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="g Measure",Arr=gMeasure)
      
        test = label_binarize(test_labels, classes=categories)
        pred = label_binarize( y_pred, classes=categories)
        
        #roc
        roc=roc_auc_score(test, pred)
    #Accuracy
    accuracy_avg.append(accuracy)
    #Precision
    macroP_avg.append(macroP)
    microP_avg.append(microP)
    #recall
    macroR_avg.append(macroR)
    microR_avg.append(microR)
    #F measure
    macroFMeasure_avg.append(fMeasure)
    microFMeasure_avg.append( m_fMeasure)
    #Roc
    roc_avg.append(roc)

def PrintDetails(metric="smtp",time=None,measure= "Accuracy",Arr=None):
    x = PrettyTable()
    kList=[1,3,5,9,15,30,45]
    x.field_names = ["Experiment No.","DataSet1"]
    print("metric",metric)
    print("time",time)
    print("measure",measure)
    tables.write("metric\t"+str(metric)+"\n")
    tables.write("time\t"+str(time)+"\n")
    tables.write("measure\t"+str(measure)+"\n")
    average=0
    for i in range(0,len(Arr)):
        x.add_row([i+1,Arr[i]]) 
        average+=Arr[i]
    x.add_row(["Average",average/len(Arr)])   
    tables.write(str(x))
    print (x)
def NBCombined(train_data, train_labels,test_data,metric):
    #prediction of NB on test data
    nb = MultinomialNB(fit_prior=False).fit(train_data, train_labels)
    if metric=="Base":
        y_pred = nb.predict(test_data)
        return y_pred
    
    prob=nb.predict_proba(test_data)
    #y_pred = classifier.predict(test_data)
    classifier =SimMeaure(k=1,metric=metric)#,termOccurance=termOccurance,docOccurance=docOccurance)  
    classifier.fit(train_data, train_labels)
    distMatrix = classifier.predict(test_data) 
    print("Dist Mat shape", len(distMatrix))
    print("Element shape", len(distMatrix[0]))
    distList=[]
    classes=nb.classes_
    predLabels=[]
    nn=10
    
    
    for tst in range(0,test_data.shape[0]):
        probNb=np.array(prob[tst])
        simAvg=[]
        simValue=np.array(distMatrix[tst])
        neigh= np.argpartition(np.array(simValue), len(simValue) - nn)[-nn:]
        neigh_labels=np.array(train_labels)[neigh]
        for cl in range(0,len(classes)):
            indices=[ii for ii in range(0,len(neigh_labels)) if neigh_labels[ii]==classes[cl]]
            avg=0
            if len(indices)>0:
                avg=np.nanmax(simValue[indices])
            simAvg.append(avg)
        
        simAvg=(simAvg/np.nanmax(simAvg))
        totalValue=np.add(probNb, simAvg)
       # totalValue=simAvg
        #print("combined value is",totalValue)
        lblInd=np.where(totalValue == np.max(totalValue))
        ind1=0
        if len(lblInd[0])>0:
            ind1=(lblInd[0])[0]
       # print(ind1)
        lbl=classes[ind1]
       # print(lbl)
        predLabels.append(lbl)      

    return predLabels
dataset='Reuters'
count=0
documents=[]
labels=[]
rawData=[]
token_dict = dict()
loadReutersData(documents=documents,labels=labels)
print(len(documents))
categories=list(set(labels))
totalDocs= len(documents)
print (totalDocs)
stopwords = stopwords.words('english')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
vocabulary = set()
term_docs=[]
terms=[]
totalDocs= len(documents)
print (len(documents))
print ("vectorize")
tfidf = TfidfVectorizer(tokenizer=tokenize1)
print ("fit transform")
tfs = tfidf.fit_transform(documents)
#sorted_scores=display_scores(tfidf, tfs)
#groups=groupData(labels,categories) 
#n=len(sorted_scores)  
arr=tfs.todense()
n=arr.shape[1]
allData=arr
#"Base","Euclidean","Manhattan","Cosine","Jaccard", "bhatta","EPairwise",
          
measures=["EPDSM","PCC","STB_SM_new","esmtp","EISC","EDice","ESP","EBLAB_SM"]
global prob
prob=[]
labels=np.array(labels)

k_splits=5
for ks in range(0,k_splits):
    prob.append([])

labels=np.array(labels)
var=0
categories=list(set(set(labels)))
categories.sort()
print(categories)
global time1
global time2
time1,time2=0,0
split=0

for met in measures:
    print("###############")
    fname='table_'+dataset+'_MNB_'+met+'_Tfidf_5Exp.txt'
    tables = open(fname, 'w')
    tables.write("No. of features\t"+str(n)+"\n")
    print("###############")
    accuracy_avg=[]
    macroP_avg=[]
    microP_avg=[]
    macroR_avg=[]
    microR_avg=[]
    macroFMeasure_avg=[]
    microFMeasure_avg=[]
    roc_avg=[]
    split=0
    if met=="smtp" or met=="esmtp" or met=="DSM":
        var1=np.var(arr,axis=0).reshape(-1,1).ravel()
        var=np.asarray(var1).ravel()
    rand_state=[0,1,10,42,30]
    time1=time.time()
    for i in range(0,k_splits):
        xTrain, xTest, yTrain, yTest = train_test_split(arr, labels, test_size = 0.3, random_state = rand_state[i],stratify=labels)
        train_data=np.array(xTrain)
        test_data=np.array(xTest)
        train_labels=yTrain
        test_labels=yTest
        classify(metric=met) 
        split+=1

    time2=time.time()
    #Accuracy
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Accuracy",Arr=accuracy_avg)
    #Precision
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Macro Precision",Arr=macroP_avg)
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Micro Precision",Arr=microP_avg)
    #Recall
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Macro Recall",Arr=macroR_avg)
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Micro Recall",Arr=microR_avg)
    #f measure
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=macroFMeasure_avg)

    #f measure
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="Micro F Measure",Arr=microFMeasure_avg)

    #roc
    PrintDetails(metric=met,time=str(timedelta(seconds=(time2-time1))),measure="ROC",Arr=roc_avg)
    tables.close()
    print("###############")



acq
crude
earn
grain
interest
money-fx
ship
trade
7691
7691
7691
vectorize
fit transform
["['acq']", "['crude']", "['earn']", "['grain']", "['interest']", "['money-fx']", "['ship']", "['trade']"]
###############
###############
Dist Mat shape 2308
Element shape 5383


<ipython-input-1-bf9528329ad5>:473: RuntimeWarning: invalid value encountered in true_divide
  simAvg=(simAvg/np.nanmax(simAvg))


Dist Mat shape 2308
Element shape 5383


<ipython-input-1-bf9528329ad5>:473: RuntimeWarning: invalid value encountered in true_divide
  simAvg=(simAvg/np.nanmax(simAvg))


Dist Mat shape 2308
Element shape 5383


...

Dist Mat shape 2308
Element shape 5383


<ipython-input-1-bf9528329ad5>:473: RuntimeWarning: invalid value encountered in true_divide
  simAvg=(simAvg/np.nanmax(simAvg))


Dist Mat shape 2308
Element shape 5383


<ipython-input-1-bf9528329ad5>:473: RuntimeWarning: invalid value encountered in true_divide
  simAvg=(simAvg/np.nanmax(simAvg))


metric EBLAB_SM
time 2:49:10.520738
measure Accuracy
+----------------+--------------------+
| Experiment No. |      DataSet1      |
+----------------+--------------------+
|       1        | 0.9402079722703639 |
|       2        | 0.9259098786828422 |
|       3        | 0.9319757365684576 |
|       4        | 0.9285095320623917 |
|       5        | 0.9380415944540728 |
|    Average     | 0.9329289428076256 |
+----------------+--------------------+
metric EBLAB_SM
time 2:49:10.520738
measure Macro Precision
+----------------+--------------------+
| Experiment No. |      DataSet1      |
+----------------+--------------------+
|       1        | 0.9124679178110717 |
|       2        | 0.9192566610346546 |
|       3        | 0.9189258013498469 |
|       4        | 0.9134809378251625 |
|       5        | 0.9238005414927897 |
|    Average     | 0.9175863719027049 |
+----------------+--------------------+
metric EBLAB_SM
time 2:49:10.520738
measure Micro Precision
+----------------+---------

In [2]:
import datetime
print(str(datetime.timedelta(seconds=33079.05312848091)))

9:11:19.053128


In [3]:
totalValue=[0, np.nan, np.nan, 0]
np.where(totalValue == np.nanmax(totalValue))

(array([0, 3], dtype=int64),)